In [38]:
### IMPORTS ###
import pandas as pd
import yt_dlp
import json
import os
from pathlib import Path
import re

import webvtt
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation
import functools

In [35]:
# help(yt_dlp.YoutubeDL)
# Detailed explanation of yt-dlp options for your use case
"""
OPTIONS NEEDED FOR YOUR REQUIREMENTS:

1. TITLE - Automatically extracted as info_dict['title']
   
2. AUTHOR - Automatically extracted as info_dict['uploader']
   
3. LINK - The URL you provide to the function
   
4. DURATION - Automatically extracted as info_dict['duration'] (in seconds)

5. SUBTITLES WITH TIMESTAMPS (Japanese & English):
   - writesubtitles: True
     Downloads manually created subtitles
   
   - writeautomaticsub: True  
     Downloads auto-generated subtitles (YouTube's automatic captions)
   
   - subtitleslangs: ['ja', 'en']
     Specifies which subtitle languages to download
     Can use 'all' to download all available languages
   
   - subtitlesformat: 'vtt'  (RECOMMENDED)
     Format for subtitles. Options: 'vtt', 'srt', 'ass', 'lrc'
     VTT (WebVTT) includes timestamps in this format:
         00:00:01.000 --> 00:00:03.000
         Subtitle text here
     
     VTT is best for your use case because:
     - Includes precise timestamps
     - Easy to parse programmatically
     - Well-supported format

6. AUDIO FILE:
   - format: 'bestaudio/best'
     Downloads the best available audio quality
   
   - postprocessors: [{
         'key': 'FFmpegExtractAudio',
         'preferredcodec': 'mp3',      # Output format (mp3, wav, m4a, etc.)
         'preferredquality': '192',    # Audio bitrate (128, 192, 256, 320)
     }]
     Converts audio to MP3 format
   
   With the subtitle timestamps, you can later extract specific 
   audio segments using ffmpeg or pydub

OTHER USEFUL OPTIONS:

- writeinfojson: True
  Saves ALL metadata as a .info.json file including:
  - Full description, tags, categories
  - View count, like count, upload date
  - All available formats and quality info
  - Comments (if getcomments: True)
  
- outtmpl: 'path/%(title)s.%(ext)s'
  Controls output filename. Available variables:
  - %(title)s - Video title
  - %(uploader)s - Channel/uploader name
  - %(id)s - Video ID
  - %(upload_date)s - Upload date (YYYYMMDD)
  - %(duration)s - Duration in seconds
  Example: 'downloads/%(uploader)s/%(title)s_%(id)s.%(ext)s'

- writethumbnail: True
  Downloads the video thumbnail image
  
- quiet: False
  Shows download progress (True hides it)

- getcomments: True (with writeinfojson: True)
  Extracts video comments and saves to JSON
"""

"\nOPTIONS NEEDED FOR YOUR REQUIREMENTS:\n\n1. TITLE - Automatically extracted as info_dict['title']\n\n2. AUTHOR - Automatically extracted as info_dict['uploader']\n\n3. LINK - The URL you provide to the function\n\n4. DURATION - Automatically extracted as info_dict['duration'] (in seconds)\n\n5. SUBTITLES WITH TIMESTAMPS (Japanese & English):\n   - writesubtitles: True\n     Downloads manually created subtitles\n\n   - writeautomaticsub: True  \n     Downloads auto-generated subtitles (YouTube's automatic captions)\n\n   - subtitleslangs: ['ja', 'en']\n     Specifies which subtitle languages to download\n     Can use 'all' to download all available languages\n\n   - subtitlesformat: 'vtt'  (RECOMMENDED)\n     Format for subtitles. Options: 'vtt', 'srt', 'ass', 'lrc'\n     VTT (WebVTT) includes timestamps in this format:\n         00:00:01.000 --> 00:00:03.000\n         Subtitle text here\n\n     VTT is best for your use case because:\n     - Includes precise timestamps\n     - Easy t

In [ ]:
def download_media_with_subtitles(url, output_dir='data'):
    """
    Download audio as MP3, subtitles, and metadata from a given URL.
    
    Args:
        url        (str): URL of the media to download
        output_dir (str): Directory to save downloaded files
    
    Returns:
        result: Information about downloaded files
    """
    
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)     
    
    # Configure yt-dlp options
    ydl_opts = {
        # Audio settings
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        
        # Output settings
        'outtmpl': f'{output_dir}/%(title)s.%(ext)s',
        
        # Subtitle settings - KEY FOR YOUR USE CASE
        'writesubtitles': True,              # Manual subs
        'writeautomaticsub': True,           # Auto-generated subs
        'subtitleslangs': ['ja'],            # Japanese (use translator to convert to en)
        'subtitlesformat': 'vtt',            # VTT format (includes timestamps)
        
        # Metadata settings
        'writeinfojson': True,               # Save full metadata as JSON
        'writethumbnail': False,             # Set True if you want thumbnails
        
        # Rate limiting to avoid 429 errors
        'sleep_interval_requests': 2,        # Sleep 2 seconds between API requests
        'sleep_interval_subtitles': 10,      # Sleep 10 seconds between subtitle downloads
        'sleep_interval': 1,                 # Sleep 1 second before each download
        
        # Progress settings
        'quiet': False,
        'no_warnings': False,
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # Extract info without downloading first to check what's available
            info = ydl.extract_info(url, download=False)

            # Display what's available
            print(f"\nTitle: {info.get('title')}")
            print(f"Author: {info.get('uploader', 'N/A')}")
            print(f"Duration: {info.get('duration', 'N/A')} seconds")
            print(f"URL: {url}\n")
            
            # Check subtitles
            manual_subs = list(info.get('subtitles', {}).keys())
            auto_subs = list(info.get('automatic_captions', {}).keys())
            print(f"Manual subtitles: {manual_subs if manual_subs else 'None'}")
            print(f"Auto-captions available for: {', '.join(auto_subs[:5])}... (and {len(auto_subs)-5} more)" if len(auto_subs) > 5 else f"Auto-captions: {auto_subs}\n")
            
            # Download everything
            print("\n===== Downloading audio, subtitles, and metadata... =====")
            print("(Adding delays between requests to avoid rate limiting...)\n")     

            ydl.download([url])
            
            result = {
                'title': info.get('title'),
                'duration': info.get('duration'),
                'uploader': info.get('uploader'),
                'upload_date': info.get('upload_date'),
                'view_count': info.get('view_count'),
                'like_count': info.get('like_count'),
                'description': info.get('description'),
                'subtitles_available': list(info.get('subtitles', {}).keys()),
                'auto_captions_available': list(info.get('automatic_captions', {}).keys()),
                'output_dir': output_dir
            }
            
            print("\nDownload complete!\n")
            return result
    except Exception as e:
        print(f"Error: {str(e)}\n")
        return None


In [50]:
# Test the function with a URL
# Replace with your desired URL
# url = "https://www.youtube.com/watch?v=1PPLx2N_YTE" # YUYU podcast test, auto gen subs
# url = "https://www.youtube.com/watch?v=hkeegM5t-lw" # Jiro podcast test, manual subs
# url = "https://open.spotify.com/track/7DyS11kB1YNrfDzQqtjmTh?si=24a7621a3a24483c" # spotify song
url = "https://www.youtube.com/watch?v=1mz-A--mANU"

# Download audio, subtitles, and metadata
result = download_media_with_subtitles(url, output_dir='japanese_media')

# Display the metadata
if result:
    #print("\\n\" + \"=\"*50")
    print("DOWNLOADED MEDIA INFORMATION\n")
    #print("=\"*50")
    print(json.dumps(result, indent=2, ensure_ascii=False))

[youtube] Extracting URL: https://www.youtube.com/watch?v=1mz-A--mANU
[youtube] 1mz-A--mANU: Downloading webpage


[youtube] Sleeping 2 seconds ...
[youtube] 1mz-A--mANU: Downloading android vr player API JSON
[youtube] Sleeping 2 seconds ...
[youtube] 1mz-A--mANU: Downloading ios downgraded player API JSON
[youtube] Sleeping 2 seconds ...
[youtube] 1mz-A--mANU: Downloading m3u8 information
[info] 1mz-A--mANU: Downloading subtitles: ja

Title: 平井 大 / Slow & Easy（Music Video）
Author: 平井 大 HIRAIDAI
Duration: 222 seconds
URL: https://www.youtube.com/watch?v=1mz-A--mANU

Manual subtitles: ['ja']
Auto-captions available for: ja, ab-ja, aa-ja, af-ja, ak-ja... (and 152 more)

===== Downloading audio, subtitles, and metadata... =====
(Adding delays between requests to avoid rate limiting...)

[youtube] Extracting URL: https://www.youtube.com/watch?v=1mz-A--mANU
[youtube] Sleeping 2 seconds ...
[youtube] 1mz-A--mANU: Downloading webpage


[youtube] Sleeping 2 seconds ...
[youtube] 1mz-A--mANU: Downloading android vr player API JSON
[youtube] Sleeping 2 seconds ...
[youtube] 1mz-A--mANU: Downloading ios downgraded player API JSON
[youtube] Sleeping 2 seconds ...
[youtube] 1mz-A--mANU: Downloading m3u8 information
[info] 1mz-A--mANU: Downloading subtitles: ja
[info] 1mz-A--mANU: Downloading 1 format(s): 251
[info] Writing video subtitles to: japanese_media/平井 大 ⧸ Slow & Easy（Music Video）.ja.vtt
[download] Sleeping 10.00 seconds ...


[download] Destination: japanese_media/平井 大 ⧸ Slow & Easy（Music Video）.ja.vtt
[download] 100% of    3.31KiB in 00:00:00 at 35.56KiB/s
[info] Writing video metadata as JSON to: japanese_media/平井 大 ⧸ Slow & Easy（Music Video）.info.json
[download] Sleeping 1.00 seconds ...
[download] Destination: japanese_media/平井 大 ⧸ Slow & Easy（Music Video）.webm
[download] 100% of    3.58MiB in 00:00:01 at 2.19MiB/s   
[ExtractAudio] Destination: japanese_media/平井 大 ⧸ Slow & Easy（Music Video）.mp3
Deleting original file japanese_media/平井 大 ⧸ Slow & Easy（Music Video）.webm (pass -k to keep)

Download complete!

DOWNLOADED MEDIA INFORMATION

{
  "title": "平井 大 / Slow & Easy（Music Video）",
  "duration": 222,
  "uploader": "平井 大 HIRAIDAI",
  "upload_date": "20181220",
  "view_count": 19151280,
  "like_count": 85267,
  "description": "平井 大の楽曲をチェック！\nhttps://DaiHirai.lnk.to/toppage\n\nFOLLOW \"HIRAIDAI\"\nWebsite: https://hiraidai.com/\nInstagram: https://www.instagram.com/hirai_dai_official/\nTwitter: https://t

In [53]:
### CREATING THE vtt PARSER ###
import re
from pathlib import Path

In [ ]:
# 1.) vtt cleaner

# Remove empty lines
# Remove WEBVTT, Kind: captions, Language: ja lines (always first 3)
# Remove all lines with "<c>" or "</c>" since these are bad format and are repeatted in 
# next line anyways

# Output to (cleaned)+path
def vtt_clean(input_file, output_file):
    input_path = Path(input_file)
    output_path = Path(output_file)

    if not input_path.exists():
        raise FileNotFoundError(F"File not found: {input_path}")

    # Read full file into content buffer
    content = input_path.read_text(encoding='utf-8')

    # Process
    re.sub(r"\n\n\n|\n\n", "\n", content)
    content = '\n'.join(
        line for line in content.split('\n') 
        if '<c>' not in line 
        and '</c>' not in line
        and 'WEBVTT' not in line
        and 'Kind: captions' not in line
        and 'Language: ja' not in line
    )

    content = content.split("\n")
    content = [item for item in content if item not in ('', ' ')]
    # for item in content:
    #     print(item)

    # Now must make json data for start_time, end_time, text
    # If the line has "-->" its a time stamp
    # sometimes 2 timestamps are back to back lines so we can't flip

    # use a sliding window:


In [98]:
vtt_clean("/home/evanxavierhu/Documents/VSCode_Local_Projects/JL_Project/Japanese_Lang_Project/japanese_media/【1hour Podcast】本名は？結婚してる？苦手なことは？YUYUについて話しました！ (Japanese Radio for Listening practice).ja.vtt", "data1.txt")

#vtt_clean("/home/evanxavierhu/Documents/VSCode_Local_Projects/JL_Project/Japanese_Lang_Project/japanese_media/【Japanese Podcast】I Got This! - Master 706 Essential Words Without Even Noticing.ja.vtt", "data2.txt")

00:00:00.160 --> 00:00:02.230 align:start position:0%
00:00:02.230 --> 00:00:02.240 align:start position:0%
はい、皆さんこんにちは。ゆの日本語
00:00:02.240 --> 00:00:04.470 align:start position:0%
はい、皆さんこんにちは。ゆの日本語
00:00:04.470 --> 00:00:04.480 align:start position:0%
ポッドキャストのお時間です。皆さん元気
00:00:04.480 --> 00:00:07.070 align:start position:0%
ポッドキャストのお時間です。皆さん元気
00:00:07.070 --> 00:00:07.080 align:start position:0%
にしていますでしょうか?今日は久しぶり
00:00:07.080 --> 00:00:09.830 align:start position:0%
にしていますでしょうか?今日は久しぶり
00:00:09.830 --> 00:00:09.840 align:start position:0%
に1時間ポッドキャストにチャレンジし
00:00:09.840 --> 00:00:14.150 align:start position:0%
に1時間ポッドキャストにチャレンジし
00:00:14.150 --> 00:00:14.160 align:start position:0%
たいと思います。テーマはゆについて、あ
00:00:14.160 --> 00:00:17.109 align:start position:0%
たいと思います。テーマはゆについて、あ
00:00:17.109 --> 00:00:17.119 align:start position:0%
、ま、つまり僕についてですね。ま、
00:00:17.119 --> 00:00:19.950 align:start position:0%
、ま、つまり僕についてですね。ま、
00:00:19.950 --> 00:00:19.960 align:start position:0%
おかげ様でYouTubeの登